In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from healpix_geo.nested import healpix_to_lonlat

In [ ]:
df3 = pd.read_parquet("cell-areas-depth-3.parquet")
df3

In [ ]:
df6 = pd.read_parquet("cell-areas-depth-6.parquet")
df6

In [ ]:
df10 = pd.read_parquet("cell-areas-depth-10.parquet")
df10

In [ ]:
df11 = pd.read_parquet("cell-areas-depth-11.parquet")
df11

In [ ]:
def plot_ratio(df, level, *, ax=None):
    import matplotlib.ticker as mtick

    _, lat = healpix_to_lonlat(
        df["cell_id"].to_numpy(), depth=level, ellipsoid="sphere"
    )

    with_lat = df.assign(lat=lat).sort_values(by="lat")
    subset = with_lat.set_index("lat").loc[0:90]

    ratio_naive = subset["areas_lazy_wgs84"] / subset["areas_wgs84"]
    ratio_exact = subset["areas_wgs84"] / subset["areas_wgs84"].max()

    new_df = pd.DataFrame(
        {"authalic mapping": ratio_exact, "lazy mapping": ratio_naive}
    )
    axis = new_df.pipe(lambda df: df * 100).plot(
        xlabel=r"$\varphi$ [$\degree$] - starting latitude on the sphere",
        ylabel="local area stretching factor",
        xticks=[0, 45, 90],
        xlim=[0, 90],
        legend=True,
        ax=ax,
    )
    axis.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=1))
    axis.grid(which="major", linestyle="--")

    if ax is not None:
        axis.set_title(f"Area stretching for level {level}")

    return axis

In [ ]:
ax = plot_ratio(df10, level=10)
ax.set_title("area distortion for refinement level 10")
ax.figure.savefig(
    "cell-area-ratios-level10.png", dpi=600, bbox_inches="tight", format="png"
)

In [ ]:
ax = plot_ratio(df11, level=11)
ax.set_title("area distortion for refinement level 11")
ax.figure.savefig(
    "cell-area-ratios-level11.png", dpi=600, bbox_inches="tight", format="png"
)

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=1, sharey=True, figsize=(10, 10))
plot_ratio(df3, level=3, ax=axes[0])
plot_ratio(df6, level=6, ax=axes[1])
plot_ratio(df10, level=10, ax=axes[2])
fig.savefig("cell-area-ratios.png", dpi=600, bbox_inches="tight", format="png")

In [ ]:
df6["areas_wgs84"].pipe(lambda s: s / s.mean() * 100).plot()